In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
def Add_Block_Matrices(Paa, Pbb):
    Pab = np.zeros(Paa.shape)
    Pba = np.zeros(Paa.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [4]:
# get boreland setup

For $N=3$ and $M=6$
These are necessary and sufficient conditions, this means 1. when they are fulfilled the matrix is N representable and 2. wheh the matrix is N-representable, they are fulfilled. It should be easy to test, by looking at a 1RDM from Li in 6 AOs\
$n_1 + n_6 = 1$\
$n_2 + n_5 = 1$\
$n_3 + n_4 = 1$\
$n_5 + n_6 > n_4$\
$n_i > 0$

In [5]:
# make Li
mol = gto.Mole()
mol.atom = f"""
    Li    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = {'Li': [[0,
                    [19.2406000, 0.0328280],
                    [2.8992000, 0.2312080],
                    [0.6534000, 0.8172380],],
                   [0,
                    [0.1776000, 1.0000000],],
                   [0,
                    [1.0000000, 1.0000000],]],
}#mol.basis = "sto-6g"
mol.spin = 1 # spins [PSE[el]-1 ] 
mol.verbose=0
mol.build()

In [6]:
# run Hartree Fock
mf = scf.RHF(mol)
mf.kernel()

np.float64(-6.601784460350542)

In [7]:
# Harvest
C = mf.mo_coeff
h1 = hf.mo_coeff.T.dot(hf.get_hcore()).dot(hf.mo_coeff)
h2 = ao2mo.kernel(mol, hf.mo_coeff)

In [8]:
# run FCI
fs = fci.FCI(mol, mf.mo_coeff)
e, ci = fs.kernel(verbose=0)

# cisolver = fci.FCI(mol, mf.mo_coeff)
# e_1, fcivec = cisolver.kernel()

In [9]:
# Preparing Data for the Natural Orbital Functionals
FCIgamma_a, FCIgamma_b = fci.direct_spin1.make_rdm1s(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIoccu_a, FCInaturalC_a = np.linalg.eigh(FCIgamma_a)
FCIoccu_b, FCInaturalC_b = np.linalg.eigh(FCIgamma_b)
FCInaturalC_a = FCInaturalC_a[:,::-1]
FCInaturalC_b = FCInaturalC_b[:,::-1]
FCIoccu_a = FCIoccu_a[::-1]
FCIoccu_b = FCIoccu_b[::-1]
# I think I did this just for the 4 index integrals being written in AO basis
FCInaturalCTT_a, FCInaturalCTT_b = FCInaturalCTT_a, FCInaturalCTT_b  #np.matmul(C,FCInaturalC_a), np.matmul(C,FCInaturalC_b)
FCInaturalCTTE = Add_Block_Matrices(FCInaturalCTT_a, FCInaturalCTT_b)
FCIoccuE = np.append(FCIoccu_a,FCIoccu_b)
for i, n  in enumerate(FCIoccuE):
    if n < 0:
        FCIoccuE[i] = 0 

In [20]:
i = np.argsort(FCIoccuE)
l = FCIoccuE[i[::-1]]
FCInaturalCTTE = FCInaturalCTTE[:,i]

In [21]:
# Test if Dennis Boreland condition comes True
print(l)
print(l[0]+l[5], l[1]+l[4], l[2]+l[3], l[4]+l[5] > l[3])

[9.99997187e-01 9.99906230e-01 9.99903418e-01 9.65822750e-05
 9.37695268e-05 2.81274825e-06]
0.9999999999999999 1.0 0.9999999999999997 True


infringe condition, how? There are conditions, we cant infringe. $0 < n_i < 1$, since otherwise the pauli principle does not hold anymore. Also $\sum_i n_i = N$, since otherwise system si not closed anymore. Moreover, the sequence comes from the condition, when we change to far the naming of elements changes but the sequence stays the same. All possible changes will be included if only non sequence changing changes are done, i.e. we have three unvreakable conditions:\
$\sum_i n_i = N$\
$0 < n_i < 1$\
$n_i > n_{i+1}$\
This means we can only move occupation around like a plastic deformation, this keeps N, and we can effectively only make the first 3 smaller and the subsequent 3 larger. Possibly like this:
$n_1 = n_1 - a$\
$n_4 = n_4 + a$\
where:\
$a < n_1 - n_2$\
For Instance:\
$a = (n_1 - n_2)/2$

Now;\
$n_i > n_{i+1}$\
$0 < n_i < 1$\
$\sum_i n_i = N$\
but:\
$n_1 + n_6 = 1-a$\
and:\
$n_3 + n_4 = 1+a$


In [22]:
print(l)
a = (l[0]-l[1])/2
print(a)
l[0] = l[0]-a
l[3] = l[3]+a
print(l)


[9.99997187e-01 9.99906230e-01 9.99903418e-01 9.65822750e-05
 9.37695268e-05 2.81274825e-06]
4.547838926222125e-05
[9.99951709e-01 9.99906230e-01 9.99903418e-01 1.42060664e-04
 9.37695268e-05 2.81274825e-06]


In [13]:
print(l[0]+l[5], l[1]+l[4], l[2]+l[3], l[4]+l[5] > l[3])

0.9999545216107377 1.0 1.0000454783892618 False


Now Rebuild the 1RDM

In [26]:
gamma = np.matmul(np.matmul(FCInaturalCTTE,np.diag(l)),FCInaturalCTTE.T)

Now follows the inversion:

In [ ]:
# eq_cons = {'type': 'eq',
#            'fun' : lambda x: x[0]-h1[0, 0]}
# # initializing the calculator
# calculator = Calculator(h2, gamma, # two body term and rdm for the ground state
#                         tools.e_rdms, tools.optimize, 1e-3, # function that calculates (energies, rdms) and optimizer 
#                         ecore=mol.energy_nuc(), norbs=h1.shape[1], nelec=mol.nelec, maximize=True, symmetrize=True, nroots=5) # parameters for the


# # run the optimizer
# ## objectiva function value changes when we add constraint
# res_LFT = calculator.optimize(np.zeros(h1.size), 
#                           method='trust-constr',
#                           options={"maxiter":500, 'disp':True, 'verbose':0, 'gtol':1e-7,},
#                           constraints=eq_cons)
# LV.append(res_LFT.fun)